In [1]:
import numpy as np

def global_softmax(x):
    """一次性算 softmax（数值稳定版本）"""
    M = np.max(x)
    exps = np.exp(x - M)
    return exps / np.sum(exps)

def chunk_softmax(x, chunk_size=2):
    """分块计算 softmax"""
    M = -np.inf   # 当前全局最大值
    S = 0.0       # 当前全局指数和

    # 先保存每个元素的值，等会用全局 M 归一化
    chunks = []
    
    for i in range(0, len(x), chunk_size):
        block = x[i:i+chunk_size]
        m = np.max(block)  # 当前块最大值
        s = np.sum(np.exp(block - m))  # 当前块指数和
        chunks.append((block, m, s))

        if m <= M:
            # 旧最大值更大 → 换基准
            S = S + s * np.exp(m - M)
        else:
            # 新最大值更大 → 旧结果要缩放
            S = S * np.exp(M - m) + s
            M = m  # 更新全局最大值
    
    # 现在有了全局 M 和 S，计算 softmax
    softmax_vals = []
    for block, m, s in chunks:
        softmax_vals.extend(np.exp(block - M) / S)

    return np.array(softmax_vals)


# ====== 测试 ======
x = np.array([1000, 1001, 999, 1002], dtype=np.float64)

print("全局 softmax:")
print(global_softmax(x))

print("\n分块 softmax (每块 2 个):")
print(chunk_softmax(x, chunk_size=2))


全局 softmax:
[0.08714432 0.23688282 0.0320586  0.64391426]

分块 softmax (每块 2 个):
[0.08714432 0.23688282 0.0320586  0.64391426]
